In [1]:
import gradio as gr

from pathlib import Path
from dotenv import load_dotenv
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
load_dotenv()

True

In [3]:
documents = []
folders = Path(".").glob("knowledge-base/*")

for folder in folders:
    if folder.is_dir():
        loader = DirectoryLoader(folder, glob="*.md", loader_cls=TextLoader)
        docs = loader.load()
        documents += docs

print(len(documents))

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

print(len(chunks))

Created a chunk of size 1088, which is longer than the specified 1000


31
123


In [4]:
db_name = "vector_db"
db_path = Path(db_name)
embeddings = OpenAIEmbeddings()

if db_path.exists():
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

store = Chroma.from_documents(chunks, persist_directory=db_name, embedding=embeddings)
print(f"Vector store created with {store._collection.count()} documents.")

Vector store created with 123 documents.


In [5]:
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm = ChatOpenAI(model_name="gpt-5")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retriever = store.as_retriever()

# create RAG-pipeline
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)


/var/folders/ny/bjrptr9d4f9c3mmxj10zzcl40000gp/T/ipykernel_25824/636416671.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [6]:
test_query = "What is insurellm? Respond in one sentence."
resp = conversation_chain.invoke({"question":test_query})
print(resp["answer"])

Insurellm is an innovative insurance technology company founded in 2015 that serves 300+ clients worldwide with four software products—Carllm, Homellm, Rellm, and Marketllm—powering auto, home, and reinsurance solutions and a consumer marketplace.


In [ ]:
def chat(message, history):
    resp = conversation_chain.invoke({"question": message})
    return resp["answer"]   

ui = gr.ChatInterface(fn=chat, type="messages", title="AI chat powered by RAG")
ui.launch(inbrowser=True)